# Textklassifikation, Teil 2

Im ersten Teil haben wir Texte aus dem Datensatz *GermEval 2018* mithilfe von Frequency-Vektoren und einem Naive-Bayes-Klassifikator klassifiziert.

Dieses Mal verwenden wir ein vortrainiertes *Word2Vec* Modell und wollen untersuchen, ob wir damit bessere Ergebnisse erzielen.

## Daten einlesen

Wie beim letzten Mal lesen wir zunächst die Daten ein.

In [ ]:
import pandas as pd

train_ds = pd.read_csv('../data/GermEval-2018/germeval2018.training.txt', sep='\t', header=None, names=['text', 'coarse', 'fine'])
test_ds = pd.read_csv('../data/GermEval-2018/germeval2018.test.txt', sep='\t', header=None, names=['text', 'coarse', 'fine'])

In [ ]:
train_ds

## Texte bereinigen

Auch die Bereinigung der Texte hatten wir schon beim letzten Mal.

In [ ]:
import re

def clean_tweet(text):
    """ Preprocess and tokenize a tweet. """
    
    # remove handles, i.e. @username
    text = re.sub('\@\w+', '', text)

    # remove hashtags, quotes, etc.
    text = re.sub('[\#"\']+', '', text)
    
    # replace hyphens with blanks
    text = text.replace('-', ' ')
    return text

In [ ]:
train_ds['text'] = train_ds['text'].map(clean_tweet)
test_ds['text'] = test_ds['text'].map(clean_tweet)

## Word2Vec-Modell laden

Die Datei `twitter-de_d100_w5_min10.bin` enthält ein mit deuschen Twitternachrichten trainiertes *Word2Vec* Modell, das wir nun laden.

In [32]:
import gensim 

w2v = gensim.models.KeyedVectors.load_word2vec_format('/home/archive/nlp/twitter-de_d100_w5_min10.bin', binary=True)

In [37]:
w2v.get_vector("Putin") 

array([-3.6704499e-01, -2.5845766e-01,  3.3164865e-01, -2.8627914e-01,
       -3.6947533e-02,  1.8208618e-01, -3.1218007e-01,  7.5630613e-02,
       -1.4815192e-01,  1.3980363e-01, -6.3882720e-01,  3.7740152e-02,
       -2.8488025e-01, -4.6945715e-01,  8.6255640e-02,  3.8550335e-01,
        2.4608003e-01,  1.8022247e-01, -6.4713076e-02,  1.1324853e+00,
       -3.5137454e-01, -2.6744670e-01, -2.8706011e-01, -2.1943483e-01,
        2.8853205e-01,  2.3831853e-01,  6.5529160e-02,  3.3113310e-01,
        7.0876971e-02, -1.5132450e-01,  2.1864030e-01,  1.0246916e-01,
        2.2849274e-01, -2.8845105e-01,  1.4559811e-01,  2.4681638e-01,
       -3.3567261e-02,  2.2350734e-01,  1.4540048e-01, -3.9889014e-01,
        1.7679639e-01,  1.1940583e-01, -5.9695488e-01,  6.2303308e-02,
        9.0350956e-02, -5.1355201e-01, -1.6860448e-01,  1.7907012e-01,
       -1.3151786e-01, -5.3585225e-01,  5.5372530e-01,  1.5263624e-01,
        1.9472370e-01, -4.2441699e-01, -1.6323459e-01, -4.7798982e-01,
      

In [31]:
w2v.most_similar("Putin")

[('Erdogan', 0.9224411249160767),
 ('Assad', 0.9186140894889832),
 ('Poroschenko', 0.9171161651611328),
 ('Hollande', 0.8956663012504578),
 ('Putins', 0.8930777907371521),
 ('Russlands', 0.8892519474029541),
 ('Syrien-Konflikt', 0.8809431195259094),
 ('Kreml', 0.8785954117774963),
 ('Obama', 0.8760547041893005),
 ('Lawrow', 0.8737961649894714)]

In [34]:
frau = w2v.get_vector("Frau")
mann = w2v.get_vector("Mann")
koenig = w2v.get_vector("König")

w2v.similar_by_vector(koenig - mann + frau)

[('Königin', 0.7747876644134521),
 ('König', 0.7370375394821167),
 ('Elisabeth', 0.7069919109344482),
 ('Kaiserin', 0.6863002181053162),
 ('Ikone', 0.6776044368743896),
 ('Doris', 0.6739816069602966),
 ('Agnes', 0.6727083921432495),
 ('Maria', 0.6692684888839722),
 ('Katharina', 0.6676202416419983),
 ('Regisseurin', 0.665666401386261)]

In [ ]:
paris = w2v.get_vector("Paris")
berlin = w2v.get_vector("Berlin")
deutschland = w2v.get_vector("Deutschland")

w2v.similar_by_vector(paris - berlin + deutschland)

In [ ]:
help(w2v)

## Aufgabe 1: Wortähnlichkeiten

Der Vorteil von Word2Vec ist, dass die generierten Embeddings die Bedeutung von Worten widerspiegeln.
Nutzen Sie die Methode `most_similar` des Modells, um ähnliche Worte für folgende Begriffe zu finden:

- Stümper
- Merkel
- Hetze

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics


def evaluate(classifier):
    """Evaluiere einen classifier auf den Testdaten"""
    predicted = classifier.predict(test_ds['text'])
    print(f"Confusion matrix:\n{metrics.confusion_matrix(test_ds['coarse'], predicted)}")
    print(f"{metrics.classification_report(test_ds['coarse'], predicted)}")
    return np.mean(predicted == test_ds['coarse'])
    

## Aufgabe 2: Wortvektoren zur Textklassifikation

Wir wollen nun untersuchen, inwiefern die Wortvektoren uns bei der Klassifikation der Tweets helfen. Die folgende Klasse 
`EmbeddingVectorizer` ordnet jedem Text den "Durchnittsvektor" seiner Worte zu. Dazu verwendet sie die Methode `get_mean_vector`.

Verwenden Sie den `EmbeddingVectorizer` und einen `LinearSVC`, um die Tweets zu klassifizieren.

Trainieren Sie den Klassifikator auf den Trainingsdaten und evaluieren Sie ihn auf den Testdaten. 
Wie gut ist Ihr Ergebnis? Vergleichen Sie Ihr Ergebnis mit dem Ergebnis vom letzten Mal und den [Ergebnissen des GermEval-2018](../data/GermEval-2018/results.pdf).

In [ ]:
from sklearn.base import BaseEstimator
import nltk
import numpy as np

class EmbeddingVectorizer(BaseEstimator):
    """Convert a collection of text documents to a matrix of vectors created from word embeddings """
    
    def __init__(self, model):
        self.model = model
        self.tokenizer = nltk.tokenize.casual.TweetTokenizer()
    
        
    def fit(self, X, y, **fit_params):
        """Nothing to do here, we use a pre-trained model. """
        return self
    
    def transform(self, raw_documents):
        """Transform documents to embedding matrix by mean of the embeddings
        of individual words.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        _X = []
        for doc in raw_documents:
            words = self.tokenizer.tokenize(doc)
            _X.append(self.model.get_mean_vector(words))
            
        X = np.array(_X)
        return X

In [ ]:
vectorizer = EmbeddingVectorizer(w2v)